In [1]:
training_file_path = '../resources/WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.data.xml'
gold_file_path =  '../resources/WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.gold.key.txt'

training_file_path = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2007/semeval2007.data.xml'
gold_file_path = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2007/semeval2007.gold.key.txt'

training_file_path_dev = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2013/semeval2013.data.xml'
gold_file_path_dev = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2013/semeval2013.gold.key.txt'
fine_senses_vocab_path = '../resources/semcor.vocab.WordNet.json'
input_vocab_path = '../resources/semcor.input.vocab.json'
input_antivocab_path = '../resources/semcor.leftout.vocab.json'
embedding_size = 32
batch_size = 64
LEARNING_RATE = 0.01
N_EPOCHS = 10
PADDING_SIZE = 50
print_model = False

In [2]:
import models
import utils
import generators

#from tensorflow.random import set_random_seed
#set_random_seed(42)
import tensorflow.keras as K
#import keras as K
import time
import os
#from tqdm import tqdm
import numpy as np
import time
import os

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
#loading dict
senses = utils.json_vocab_reader(fine_senses_vocab_path)
inputs, antivocab = utils.json_vocab_reader(input_vocab_path, input_antivocab_path)
output_vocab = utils.vocab_merge(senses, inputs)
reverse_output_vocab =  dict((v, k) for k, v in output_vocab.items())

K.backend.clear_session()


# Build Model

In [4]:
model = models.Basic(test=2)
BasicModelNetwork = model.build(vocab_size = len(output_vocab),
                                embedding_size = embedding_size,
                                hidden_size = 32,
                                PADDING_SIZE = PADDING_SIZE,
                                LEARNING_RATE = LEARNING_RATE,
                                INPUT_DROPOUT = 0.2,
                                LSTM_DROPOUT = 0.45,
                                RECURRENT_DROPOUT = 0.35,
                                N_EPOCHS = N_EPOCHS)

if print_model is True:
    BasicModelNetwork.summary()


Creating KERAS model
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
train_generator = generators.Basic(batch_size = 64,
                                training_file_path = training_file_path,
                                gold_file_path = gold_file_path,
                                antivocab = antivocab,
                                output_vocab = output_vocab,
                                PADDING_SIZE = PADDING_SIZE)

validation_generator = generators.Basic(batch_size = 64,
                                         training_file_path = training_file_path_dev,
                                         gold_file_path = gold_file_path_dev,
                                         antivocab = antivocab,
                                         output_vocab = output_vocab,
                                         PADDING_SIZE = PADDING_SIZE)


In [6]:
if not os.path.exists('../resources/logging'):
    os.mkdir('../resources/logging')
model_name = time.strftime('%Y-%m-%d_%H:%M:%S_%z')
cbk = K.callbacks.TensorBoard('../resources/logging/keras_model_'+model_name)

early_stopping = K.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=3,
                              verbose=2, mode='auto')
csv_logger = K.callbacks.CSVLogger('../resources/logging/keras_model_'+model_name+'.log')
model_checkpoint = K.callbacks.ModelCheckpoint(filepath = '../resources/logging/keras_model_'+model_name+'.h5',
                                               monitor='val_precision',
                                               verbose=2,
                                               save_best_only=True,
                                               save_weights_only=True,
                                               mode='auto', period=1)





In [7]:
epochs = 4
from tqdm import tqdm
c1, c2 = [0,0], [0,0]

In [8]:


for epoch in range(1, epochs+1):
    with tqdm(total=epochs) as epoch_bar:
        c1, c2 = [0,0], [0,0]
        epoch_bar.set_description("Epoch {}/{}\tloss: {:.3f} \tacc: {:.3f}\tval_loss: {:.3f} \tval_acc: {:.3f}".format(epoch, epochs, *c1, *c2))
        for batch_x, batch_y in train_generator.__getitem__():
            c1 = BasicModelNetwork.train_on_batch(batch_x, batch_y)
            epoch_bar.set_description("Epoch {}/{}\tloss: {:.3f} \tacc: {:.3f}\tval_loss: {:.3f} \tval_acc: {:.3f}".format(epoch, epochs, *c1, *c2))

        for batch_x, batch_y in validation_generator.__getitem__():
            c2 = BasicModelNetwork.test_on_batch(batch_x, batch_y)
            epoch_bar.set_description("Epoch {}/{}\tloss: {:.3f} \tacc: {:.3f}\tval_loss: {:.3f} \tval_acc: {:.3f}".format(epoch, epochs, *c1, *c2))


Epoch 1/4	loss: 11.449 	acc: 0.653	val_loss: 11.248 	val_acc: 0.555:   0%|          | 0/4 [00:29<?, ?it/s]
Epoch 2/4	loss: 10.280 	acc: 0.653	val_loss: 9.580 	val_acc: 0.555:   0%|          | 0/4 [00:20<?, ?it/s]
Epoch 3/4	loss: 7.508 	acc: 0.653	val_loss: 7.142 	val_acc: 0.555:   0%|          | 0/4 [00:20<?, ?it/s]
Epoch 4/4	loss: 4.314 	acc: 0.653	val_loss: 4.697 	val_acc: 0.555:   0%|          | 0/4 [00:29<?, ?it/s]


In [ ]:
for epoch in range(epochs):
    with tqdm(total=len(train_generator)) as pbar:
        for batch_x, batch_y in train_generator.__getitem__():
            c1 = BasicModelNetwork.train_on_batch(batch_x, batch_y)
            pbar.set_description("Epoch {}/{}\tloss: {:.3f} \tacc: {:.3f}".format(epoch, epochs, *c1))

    with tqdm(total=len(validation_generator)) as pbar:
        for batch_x, batch_y in validation_generator.__getitem__():
            c2 = BasicModelNetwork.test_on_batch(batch_x, batch_y)
            pbar.set_description("Epoch {}/{}\tval_loss: {:.3f} \tval_acc: {:.3f}".format(epoch, epochs, *c2))


Epoch 0/4	val_loss: 4.267 	val_acc: 0.493:   0%|          | 0/4 [00:05<?, ?it/s]

In [ ]:
with tqdm(total=len(train_generator)) as pbar:
    
    for batch_x, batch_y in train_generator.__getitem__():
        c1 = BasicModelNetwork.train_on_batch(batch_x, batch_y)
        pbar.set_description("val: {:.1f} \tacc: {:.1f}".format(*c1))

In [ ]:
print("loss: {}, acc: {}".format(*c1))

In [ ]:
from tqdm import trange
from time import sleep
lv = True
ns = True
for i in trange(2, desc='outer0 loop', leave=True):
    for j in trange(4, desc='inner1 loop', leave=lv, position=ns):
        for k in trange(100, desc='inner2 loop', leave=lv, position=ns):
            sleep(0.01)

In [ ]:
from tqdm import trange
from time import sleep
t = trange(train_generator.__getitem__(), desc='Bar desc', leave=True)
for i in t:
    t.set_description("Bar desc (file %i)" % i)
    sleep(0.01)


In [ ]:
for i in t:
    t.set_description("Bar desc (file %i)" % i)
    sleep(0.01)

In [ ]:
generator = (3 * n for n in range(length))  # just doing something random
for n in tqdm(generator, total=length):
    pass

In [ ]:
x = train_generator.__getitem__()

In [ ]:
train_generator()

In [ ]:

for (i, j) in iter(x):
    print(i.shape, j.shape)


In [ ]:
for i in range(epoch):
    for j in range(len(data_y) // batchsize):
        batch_x = data_x[i * batchsize: (i + 1) * batchsize]
        batch_y = data_y[i * batchsize: (i + 1) * batchsize]
        c1 = model.train_on_batch(batch_x, batch_y)
        c2 = model.test_on_batch(batch_x, batch_y)
        print('train', c1, '///', 'test', c2)

In [ ]:
[reverse_output_vocab[e] for e in i[0,:]]

In [ ]:
next(iter(train_generator))

In [ ]:
BasicModelNetwork.fit_generator(next(iter(train_generator)), 
                                steps_per_epoch=2,
                                epochs=5, 
                                verbose=1,
                                callbacks=[cbk, csv_logger, early_stopping],
                                validation_data=next(iter(validation_generator)),
                                validation_steps=2,
                                class_weight=None,
                                max_queue_size=10,
                                workers=-1, 
                                use_multiprocessing=False,
                                shuffle=False)

In [ ]:
BasicModelNetwork.fit_generator(train_generator.__getitem__, 
                                epochs=5, 
                                verbose=1,
                                callbacks=[cbk, csv_logger, early_stopping],
                                validation_data=validation_generator.__getitem__,
                                class_weight=None,
                                max_queue_size=10,
                                workers=-1, 
                                use_multiprocessing=False,
                                shuffle=False)

In [ ]:
[3,3].shape

In [ ]:
train_generator.__getitem__().shape

In [ ]:
BasicModelNetwork.fit_generator(train_generator.__getitem__(), 
                                steps_per_epoch=2,
                                epochs=5, 
                                verbose=1,
                                callbacks=[cbk, csv_logger, early_stopping],
                                validation_data=validation_generator.__getitem__(),
                                validation_steps=2,
                                class_weight=None,
                                max_queue_size=10,
                                workers=-1, 
                                use_multiprocessing=False,
                                shuffle=False)

In [ ]:
train_generator

In [ ]:
train_generator.__len__()

In [ ]:
len(train_generator)

In [ ]:
for i, j in train_generator:
    print(i)
    break

In [ ]:
q=0
for x,y in tqdm(train_generator):
    q+=1

In [ ]:
import numpy as np
train_generator = model.prepare_sentence_batch(batch_size = batch_size,
                                                training_file_path = training_file_path,
                                                gold_file_path = gold_file_path,
                                                antivocab = antivocab,
                                                output_vocab = output_vocab,
                                                PADDING_SIZE = PADDING_SIZE)


for x,y in tqdm(train_generator):
    pass

In [ ]:
filename = 'testing.npy'

with open(filename, 'wb') as f:
    


In [ ]:
x.append

In [ ]:
batch = model.prepare_sentence_batch(batch_size = 64,
                               training_file_path = '../resources/WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.data.xml',
                               gold_file_path =  '../resources/WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.gold.key.txt',
                               antivocab = antivocab,
                               output_vocab = output_vocab)

In [ ]:
# for x,y in batch:
#     break

In [ ]:

y.reshape(*(y).shape,1)

In [ ]:
np.expand_dims(y, axis=-1)

In [ ]:
c = 0
for x, y in tqdm(batch):
    if c==10:
        break
    c+=1
    print(c, x.shape, y.shape)


In [ ]:
from sklearn.utils import shuffle


In [ ]:
x = np.zeros((3,4))

In [ ]:
y = np.ones((3,4, 1))

In [ ]:
tuple(shuffle(x, y))[0]

In [ ]:
for i, j in tqdm(validation_generator):
    print(i.shape)

In [ ]:
j.shape

In [ ]:
training_file_path_dev = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2007/semeval2007.data.xml'
gold_file_path_dev = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2007/semeval2007.gold.key.txt'

In [ ]:
validation_generator = model.prepare_sentence_batch(batch_size = 64,
                                                     training_file_path = training_file_path_dev,
                                                     gold_file_path = gold_file_path_dev,
                                                     antivocab = antivocab,
                                                     output_vocab = output_vocab,
                                                     PADDING_SIZE = PADDING_SIZE)

In [ ]:
for i, j in validation_generator:
    print(i.shape)
    

In [ ]:
for ind, row in enumerate(i):
    print(ind, "\t", row[0:5])

In [ ]:
def count(iter):
    try:
        return len(iter)
    except TypeError:
        return sum(1 for _ in iter)

In [ ]:
#count(train_generator)

In [ ]:
306//64

In [ ]:
from nltk.corpus import wordnet as wn

In [ ]:
reverse_output_vocab[24].split(":")[1]

In [ ]:
dir(wn)

In [ ]:
dir()

In [ ]:
[reverse_output_vocab[q] for q in i[2,:]]

In [ ]:
from keras.layers import Input, concatenate, Dense
from keras.models import Model
from keras.optimizers import Adam
import numpy as np

def data_gen(top_dim, bot_dim):
    """
    Generator to yield inputs and their labels in batches.
    """
    batch_size = 16
    while True:
        batch_imgs = []
        batch_labels = []
        for i in range(batch_size):
            # Create random arrays
            rand_pix = np.random.randint(100, 256)
            top_img = np.full(top_dim, rand_pix)
            bot_img = np.full(bot_dim, rand_pix)

            # Set a label
            label = np.random.choice([0, 1])

            batch_imgs.append([top_img, bot_img])
            batch_labels.append(label)

        yield batch_imgs, batch_labels
def data_gen(top_dim, bot_dim):
    """
    Generator to yield batches of two inputs (per sample) with shapes top_dim and 
    bot_dim along with their labels.
    """
    batch_size = 264
    while True:
        top_batch = []
        bot_batch = []
        batch_labels = []
        for i in range(batch_size):
            # Create random arrays
            rand_pix = np.random.randint(100, 256)
            top_img = np.full(top_dim, rand_pix)
            bot_img = np.full(bot_dim, rand_pix)

            # Set a label
            label = np.random.choice([0, 1])
            batch_labels.append(label)

            # Pack each input image separately
            top_batch.append(top_img)
            bot_batch.append(bot_img)

        yield [np.array(top_batch), np.array(bot_batch)], np.array(batch_labels)

def get_compiled_model(top_dim, bot_dim):
    """
    Return a two input one output model.
    """
    # Get the top and bottom networks
    top = get_part_model(top_dim)
    bot = get_part_model(bot_dim)

    # Prepare inputs and unify the top and the bottom networks
    inp_top = Input(shape=top_dim)
    inp_bot = Input(shape=bot_dim)
    fusion = concatenate([top(inp_top), bot(inp_bot)])

    predictions = Dense(1, activation='sigmoid')(fusion)

    # Final full model
    model = Model([inp_top, inp_bot], outputs=predictions)

    model.compile(optimizer=Adam(lr=0.001),
                  loss='binary_crossentropy',
                  metrics=["accuracy"])

    return model




In [ ]:
top_dim, bot_dim = (16, 16, 1), (16, 16, 3)
model = get_compiled_model(top_dim, bot_dim)
my_gen = data_gen(top_dim, bot_dim)
model.fit_generator(my_gen, steps_per_epoch=10)